In [5]:
import boto3
import pandas as pd; pd.set_option('display.max_column', 100)
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from sklearn.impute import SimpleImputer

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import recall_score, classification_report

## define bucket in which you are trying to reach
s3 = boto3.resource('s3')
bucket_name = 'daltondencklau-data445-bucket'
bucket = s3.Bucket(bucket_name)

## define csv file to read in the bucket
file_key= 'framingham.csv'

## syntax to allow us to read the file
bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## reading the data file
heart = pd.read_csv(file_content_stream)
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [16]:
## defining input and target variables
x = heart[['age', 'currentSmoker', 'totChol', 'BMI', 'heartRate']]
y = heart['TenYearCHD']

In [17]:
## splitting the data into training and testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y)

In [18]:
x_train.describe()

,age,currentSmoker,totChol,BMI,heartRate
count,3390.000000,3390.000000,3349.000000,3378.000000,3389.000000
mean,49.644248,0.484071,237.230218,25.825033,75.865152
std,8.569900,0.499820,44.329707,4.084840,12.051779
min,33.000000,0.000000,107.000000,15.540000,44.000000
25%,42.000000,0.000000,206.000000,23.080000,68.000000
50%,49.000000,0.000000,234.000000,25.385000,75.000000
75%,56.000000,1.000000,264.000000,28.040000,83.000000
max,70.000000,1.000000,696.000000,56.800000,143.000000


In [19]:
## defining the imputer
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')

## fitting the imputer on the training dataset
imputer.fit(x_train)

## imputing the missing values
x_train_imp = pd.DataFrame(imputer.transform(x_train), columns = x_train.columns)
x_test_imp = pd.DataFrame(imputer.transform(x_test), columns = x_test.columns)

In [20]:
## transforming variables to a 0-1 scale
scaler = MinMaxScaler()

x_train_imp = pd.DataFrame(scaler.fit_transform(x_train_imp), columns = x_train_imp.columns)
x_test_imp = pd.DataFrame(scaler.fit_transform(x_test_imp), columns = x_test_imp.columns)

### Model Builiding

In [22]:
## buidling models
logit_md = LogisticRegression().fit(x_train_imp, y_train)
rf_md = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(x_train_imp, y_train)

## predicting on the testing dataset
logit_preds = logit_md.predict_proba(x_test_imp)[:, 1]
rf_preds = rf_md.predict_proba(x_test_imp)[:, 1]

## changing liklihoods to labels
logit_labels = np.where(logit_preds < 0.1, 0, 1)
rf_labels = np.where(rf_preds < 0.1, 0, 1)

## printing classification report LOGIT
print('Logistic Regression:')
print(classification_report(y_test, logit_labels))

## printing classification report RF
print('Random Forest:')
print(classification_report(y_test, rf_labels))

Logistic Regression:
              precision    recall  f1-score   support

           0       0.95      0.29      0.44       719
           1       0.19      0.91      0.31       129

    accuracy                           0.38       848
   macro avg       0.57      0.60      0.38       848
weighted avg       0.83      0.38      0.42       848

Random Forest:
              precision    recall  f1-score   support

           0       0.95      0.30      0.46       719
           1       0.19      0.91      0.31       129

    accuracy                           0.39       848
   macro avg       0.57      0.60      0.38       848
weighted avg       0.83      0.39      0.43       848



Based on these results, I would use the Imputer with the Random Forest Model because it received a higher recall score copared to the Logistic Regression model.